<a href="https://colab.research.google.com/github/al-025/atlas-open-data/blob/main/Everything_Open_ATLAS_Open_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install required packages - if you see any warnings/errors try restarting the session and running this cell again
import sys
!pip install --upgrade --user pip
!pip install -U numpy==2.0.0 pandas==2.2.2 uproot==5.3.9 matplotlib==3.9.0 lmfit==1.3.1 awkward-pandas==2023.8.0 aiohttp==3.9.5 requests==2.32.3 vector --user

In [ ]:
import uproot # for reading .root files
import math # for mathematical functions such as square root
import awkward as ak # for handling complex and nested data structures efficiently
import numpy as np # # for numerical calculations such as histogramming
import matplotlib.pyplot as plt # for plotting
from matplotlib.ticker import MaxNLocator,AutoMinorLocator # for minor ticks
from lmfit.models import PolynomialModel, GaussianModel # for the signal and background fits
import vector #to use vectors
import requests # for HTTP access
import aiohttp # HTTP client support
import pandas as pd # convenient data structure
from itertools import chain # for quickly flattening pandas dataframe columns

In [ ]:
# ATLAS Open Data directory
path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/" # web address
samples_list = ['data_A','data_B','data_C','data_D'] # add if you want more data
file_suffix = ".GamGam.root"
tree_type = ":mini"
GeV = 1000 # scaling factor to account for units used in data files

In [ ]:
data_15G_path = path + samples_list[3] + file_suffix

# Accessing the file from the online database
with uproot.open(data_15G_path + tree_type) as t:
	tree = t

# The number of entries in the tree can be viewed
print("The number of entries in the tree are:", tree.num_entries)

# All the information stored in the tree can be viewed using the .keys() method.
print("The information stored in the tree is:", tree.keys())

# For ease of access, we can import the data into a Pandas DataFrame
df = tree.arrays(library='pd')

In [ ]:
# Grab a variable from the dataframe by indexing into df with that variable name:
df['jet_n']

In [ ]:
def make_hist(x,bin_size,xmin=None,xmax=None,xlabel='',ylabel='Events'):
  xmin = xmin or np.min(x)
  xmax = xmax or np.max(x)
  bin_edges = np.arange(start=xmin, # The interval includes this value
                    	stop=xmax+bin_size, # The interval doesn't include this value
                    	step=bin_size ) # Spacing between values
  bin_centres = np.arange(start=xmin+bin_size/2, # The interval includes this value
                      	stop=xmax+bin_size/2, # The interval doesn't include this value
                      	step=bin_size ) # Spacing between values

  data_x,_ = np.histogram(x,
                      	bins=bin_edges ) # histogram the data
  data_x_errors = np.sqrt(data_x)

  # *************
  # Main plot
  # *************
  main_axes = plt.gca() # get current axes

  # plot the data points
  main_axes.errorbar(x=bin_centres, y=data_x, yerr=data_x_errors,
                  	fmt='ko', # 'k' means black and 'o' is for circles
                  	label='Data')

  # set the x-limit of the main axes
  main_axes.set_xlim( left=xmin, right=xmax )

  # separation of x axis minor ticks
  main_axes.xaxis.set_minor_locator( AutoMinorLocator() )

  # set the axis tick parameters for the main axes
  main_axes.tick_params(which='both', # ticks on both x and y axes
                      	direction='in', # Put ticks inside and outside the axes
                      	top=True, # draw ticks on the top axis
                      	right=True ) # draw ticks on right axis

  # x-axis label
  main_axes.set_xlabel(xlabel,
                  	fontsize=13, x=1, horizontalalignment='right' )

  # write y-axis label for main axes
  main_axes.set_ylabel(ylabel,
                      	y=1, horizontalalignment='right')

  # set y-axis limits for main axes
  main_axes.set_ylim( bottom=0, top=np.amax(data_x)*1.6 )

  # add minor ticks on y-axis for main axes
  main_axes.yaxis.set_minor_locator( AutoMinorLocator() )

  # draw the legend
  main_axes.legend( frameon=False ); # no box around the legend

# convenient function for quickly flattening a column from a dataframe
def flatten_col(col):
  return pd.Series(list(chain.from_iterable(col)))

In [ ]:
# To make a histogram of a variable, simply pass it into the make_hist function together with the desired binnings and axis labels, eg:
# make_hist(df['met'], bin_size=10, xmin=0, xmax=200,
#           xlabel='MET [GeV]', ylabel='Events / 10 GeV')

make_hist(df['jet_n'], bin_size=1, xmin=0, xmax=10,
          xlabel='Number of jets', ylabel='Events')

In [ ]:
# Note that some variables have multiple values per event (one for each corresponding object present) - these need to be flattened prior to histogramming, eg:
make_hist(flatten_col(df['jet_phi']), bin_size=0.1, xlabel='jet $\phi$')

In [ ]:
# Try making histograms for some other variables here, eg lep_pt or photon_E




---

# Recreating the Higgs Boson discovery plot

In [ ]:
def result_plot(df, bin_size=3, xmin=100, xmax=160, fraction=1):
  bin_edges = np.arange(start=xmin, # The interval includes this value
                      stop=xmax+bin_size, # The interval doesn't include this value
                      step=bin_size ) # Spacing between values
  bin_centres = np.arange(start=xmin+bin_size/2, # The interval includes this value
                        stop=xmax+bin_size/2, # The interval doesn't include this value
                        step=bin_size ) # Spacing between values

  data_x,_ = np.histogram(ak.to_numpy(df['mass']),
                              bins=bin_edges ) # histogram the data
  data_x_errors = np.sqrt( data_x ) # statistical error on the data

  # data fit
  polynomial_mod = PolynomialModel( 4 ) # 4th order polynomial
  gaussian_mod = GaussianModel() # Gaussian

  # set initial guesses for the parameters of the polynomial model
  # c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
  pars = polynomial_mod.guess(data_x, # data to use to guess parameter values
                              x=bin_centres, c0=data_x.max(), c1=0,
                              c2=0, c3=0, c4=0 )

  # set initial guesses for the parameters of the Gaussian model
  pars += gaussian_mod.guess(data_x, # data to use to guess parameter values
                          x=bin_centres, amplitude=100,
                          center=125, sigma=2 )

  model = polynomial_mod + gaussian_mod # combined model

  # fit the model to the data
  out = model.fit(data_x, # data to be fit
                  pars, # guesses for the parameters
                  x=bin_centres, weights=1/data_x_errors ) #ASK

  # background part of fit
  params_dict = out.params.valuesdict() # get the parameters from the fit to data
  c0 = params_dict['c0'] # c0 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
  c1 = params_dict['c1'] # c1 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
  c2 = params_dict['c2'] # c2 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
  c3 = params_dict['c3'] # c3 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
  c4 = params_dict['c4'] # c4 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4

  # get the background only part of the fit to data
  background = c0 + c1*bin_centres + c2*bin_centres**2 + c3*bin_centres**3 + c4*bin_centres**4

  # data fit - background fit = signal fit
  signal_x = data_x - background

  # *************
  # Main plot
  # *************
  plt.axes([0.1,0.3,0.85,0.65]) # left, bottom, width, height
  main_axes = plt.gca() # get current axes

  # plot the data points
  main_axes.errorbar(x=bin_centres, y=data_x, yerr=data_x_errors,
                  fmt='ko', # 'k' means black and 'o' means circles
                  label='Data' )

  # plot the signal + background fit
  main_axes.plot(bin_centres, # x
              out.best_fit, # y
              '-r', # single red line
              label='Sig+Bkg Fit ($m_H=125$ GeV)' )

  # plot the background only fit
  main_axes.plot(bin_centres, # x
              background, # y
              '--r', # dashed red line
              label='Bkg (4th order polynomial)' )

  # set the x-limit of the main axes
  main_axes.set_xlim( left=xmin, right=xmax )

  # separation of x-axis minor ticks
  main_axes.xaxis.set_minor_locator( AutoMinorLocator() )

  # set the axis tick parameters for the main axes
  main_axes.tick_params(which='both', # ticks on both x and y axes
                      direction='in', # Put ticks inside and outside the axes
                      top=True, # draw ticks on the top axis
                      labelbottom=False, # don't draw tick labels on bottom axis
                      right=True ) # draw ticks on right axis

  # write y-axis label for main
  main_axes.set_ylabel('Events / '+str(bin_size)+' GeV',
                      horizontalalignment='right')

  # set the y-axis limit for the main axes
  main_axes.set_ylim( bottom=0, top=np.amax(data_x)*1.1 )

  # set minor ticks on the y-axis of the main axes
  main_axes.yaxis.set_minor_locator( AutoMinorLocator() )

  # avoid displaying y=0 on the main axes
  main_axes.yaxis.get_major_ticks()[0].set_visible(False)

  # Add text 'ATLAS Open Data' on plot
  plt.text(0.2, # x
          0.92, # y
          'ATLAS Open Data', # text
          transform=main_axes.transAxes, # coordinate system used is that of main_axes
          fontsize=13 )

  # Add text 'for education' on plot
  plt.text(0.2, # x
          0.86, # y
          'for education', # text
          transform=main_axes.transAxes, # coordinate system used is that of main_axes
          style='italic',
          fontsize=8 )

  lumi = 36.1
  lumi_used = str(lumi*fraction) # luminosity to write on the plot
  plt.text(0.2, # x
          0.8, # y
          '$\sqrt{s}$=13 TeV,$\int$L dt = '+lumi_used+' fb$^{-1}$', # text
          transform=main_axes.transAxes ) # coordinate system used is that of main_axes

  # Add a label for the analysis carried out
  plt.text(0.2, # x
          0.74, # y
          r'$H \rightarrow \gamma\gamma$', # text
          transform=main_axes.transAxes ) # coordinate system used is that of main_axes

  # draw the legend
  main_axes.legend(frameon=False, # no box around the legend
                  loc='lower left' ) # legend location


  # *************
  # Data-Bkg plot
  # *************
  plt.axes([0.1,0.1,0.85,0.2]) # left, bottom, width, height
  sub_axes = plt.gca() # get the current axes

  # set the y axis to be symmetric about Data-Background=0
  sub_axes.yaxis.set_major_locator( MaxNLocator(nbins='auto',
                                              symmetric=True) )

  # plot Data-Background
  sub_axes.errorbar(x=bin_centres, y=signal_x, yerr=data_x_errors,
                  fmt='ko' ) # 'k' means black and 'o' means circles

  # draw the fit to data
  sub_axes.plot(bin_centres, # x
              out.best_fit-background, # y
              '-r' ) # single red line

  # draw the background only fit
  sub_axes.plot(bin_centres, # x
              background-background, # y
              '--r' )  # dashed red line

  # set the x-axis limits on the sub axes
  sub_axes.set_xlim( left=xmin, right=xmax )

  # separation of x-axis minor ticks
  sub_axes.xaxis.set_minor_locator( AutoMinorLocator() )

  # x-axis label
  sub_axes.set_xlabel(r'di-photon invariant mass $\mathrm{m_{\gamma\gamma}}$ [GeV]',
                      x=1, horizontalalignment='right',
                      fontsize=13 )

  # set the tick parameters for the sub axes
  sub_axes.tick_params(which='both', # ticks on both x and y axes
                      direction='in', # Put ticks inside and outside the axes
                      top=True, # draw ticks on the top axis
                      right=True ) # draw ticks on right axis

  # separation of y-axis minor ticks
  sub_axes.yaxis.set_minor_locator( AutoMinorLocator() )

  # y-axis label on the sub axes
  sub_axes.set_ylabel( 'Events-Bkg' )


  # Generic features for both plots
  main_axes.yaxis.set_label_coords( -0.09, 1 ) # x,y coordinates of the y-axis label on the main axes
  sub_axes.yaxis.set_label_coords( -0.09, 0.5 ) # x,y coordinates of the y-axis label on the sub axes

# This function calculates the invariant mass of the 2-photon state
def calc_mass(photon_pt, photon_eta, photon_phi, photon_e):
    p4 = vector.zip({"pt": photon_pt, "eta": photon_eta, "phi": photon_phi, "e": photon_e})
    invariant_mass = (p4[:, 0] + p4[:, 1]).M # .M calculates the invariant mass
    return invariant_mass/GeV


In [ ]:
# Here we cumulate data from all periods to get better statistics
variables = ["photon_pt","photon_eta","photon_phi","photon_E",
             "photon_isTightID","photon_etcone20"]

all_data = []

# Loop over each file
for val in samples_list:

    # Print which sample is being processed
    print('Processing '+val+' samples')

    fileString = path + val + file_suffix # file name to open

    # Open file
    with uproot.open(fileString + tree_type) as t:
        tree = t

    numevents = tree.num_entries

    # Calculate the invariant mass and add it into the data structure
    for data in tree.iterate(variables, library="ak", entry_stop=numevents*1):
        data['mass'] = calc_mass(data['photon_pt'], data['photon_eta'], data['photon_phi'], data['photon_E'])

        # Append data to the whole sample data list
        all_data.append(data)

# Convert back to DataFrame
all_data = ak.to_dataframe(ak.concatenate(all_data))

In [ ]:
# Let's create the final result plot without making any selections on the data and see how it looks
result_plot(all_data)

In [ ]:
# Need to apply some selections so that we can see the signal 'bump' over the background

# Cut on the photon reconstruction quality
def cut_photon_reconstruction(photon_isTightID):
    # Only the events which have True for both photons are kept
    return (photon_isTightID[:,0]==True) & (photon_isTightID[:,1]==True)

# Cut on the transverse momentum
def cut_photon_pt(photon_pt):
   # Only the events where photon_pt[0] > 40 GeV and photon_pt[1] > 30 GeV are kept
    return (photon_pt[:,0] > 40*GeV) & (photon_pt[:,1] > 30*GeV)

# Cut on the energy isolation
def cut_isolation_pt(photon_ptcone20):
    # Only the events where the isolation transverse energy is less than 4 GeV are kept
    return (photon_ptcone20[:,0] < 4*GeV) & (photon_ptcone20[:,1] < 4*GeV)

# Cut on the pseudorapidity in barrel/end-cap transition region
def cut_photon_eta_transition(photon_eta):
    # Only the events where modulus of photon_eta is outside the range 1.37 to 1.52 are kept
    abs_eta = np.abs(photon_eta)
    condition_0 = (abs_eta[:, 0] < 1.37) | (abs_eta[:, 0] > 1.52)
    condition_1 = (abs_eta[:, 1] < 1.37) | (abs_eta[:, 1] > 1.52)
    return condition_0 & condition_1


In [ ]:
# Now reduce our dataset using these cuts by indexing all_data with these conditions
selected_data = all_data
print('Initial events:',len(selected_data))

selected_data = all_data[ cut_photon_reconstruction(all_data['photon_isTightID']).reindex(selected_data.index,level='entry') ]
print('After reconstruction cut:',len(selected_data))

selected_data = selected_data[ cut_photon_pt(selected_data['photon_pt']).reindex(selected_data.index,level='entry') ]
print('After pt cut:',len(selected_data))

selected_data = selected_data[ cut_isolation_pt(selected_data['photon_etcone20']).reindex(selected_data.index,level='entry') ]
print('After isolation cut:',len(selected_data))

selected_data = selected_data[ cut_photon_eta_transition(selected_data['photon_eta']).reindex(selected_data.index,level='entry') ]
print('After eta cut:',len(selected_data))


In [ ]:
result_plot(selected_data)